In [1]:
from nltk.corpus import twitter_samples

In [2]:
# loading data

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
neutral = twitter_samples.strings('tweets.20150430-223406.json')

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [3]:
# Lemmatization

from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [4]:
# Cleaning

import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [5]:
# Stop words and tokenization

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tweet_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
neutral_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
for tokens in neutral_tweet_tokens:
    neutral_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [39]:
# Extracting tokens

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
neutral_tokens_for_model = get_tweets_for_model(neutral_cleaned_tokens_list)

In [40]:
# test train partition

import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

neutral_dataset = [(tweet_dict, "Neutral")
                     for tweet_dict in neutral_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7500]
test_data = dataset[7500:]

In [41]:
dataset

[({':)': True,
   'album': True,
   'ask': True,
   'buy': True,
   "i'm": True,
   'suppose': True},
  'Positive'),
 ({'...': True,
   ':(': True,
   'cross': True,
   'diplomacy': True,
   'double': True,
   'except': True,
   'fit': True,
   'fraud': True,
   'lie': True,
   'nothing': True,
   'survival': True,
   'tolerant': True,
   'zero': True},
  'Negative'),
 ({':(': True,
   'hello': True,
   'palembang': True,
   'please': True,
   'say': True,
   'viners': True},
  'Negative'),
 ({':)': True,
   'back': True,
   'hubby': True,
   'play': True,
   'pls': True,
   'queen': True,
   'tune': True,
   'u': True},
  'Positive'),
 ({'#choiceinternationalartist': True,
   '#superjunior': True,
   '#teenchoice': True,
   ':)': True,
   'heechul': True,
   'oppa': True},
  'Positive'),
 ({'1': True,
   '2': True,
   ':(': True,
   'away': True,
   'feeling': True,
   'first': True,
   'game': True,
   'get': True,
   'good': True,
   'mtg': True,
   'playables': True,
   'round': Tr

In [42]:
# NaiveBayes Classifier model

from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy: 0.9972
Most Informative Features
                      :( = True           Negati : Positi =   2236.4 : 1.0
                      :) = True           Positi : Negati =   1058.4 : 1.0
                follower = True           Positi : Negati =     36.9 : 1.0
                     sad = True           Negati : Positi =     27.0 : 1.0
                     bam = True           Positi : Negati =     23.8 : 1.0
                     x15 = True           Negati : Positi =     18.0 : 1.0
               community = True           Positi : Negati =     17.3 : 1.0
                followed = True           Negati : Positi =     14.5 : 1.0
                    miss = True           Negati : Positi =     12.9 : 1.0
                     ugh = True           Negati : Positi =     12.6 : 1.0
None


In [43]:
# Saving model

import pickle

filename = 'finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [46]:
from nltk.tokenize import word_tokenize

custom_tweet = "hurt."
custom_tokens = remove_noise(word_tokenize(custom_tweet))

result = classifier.classify(dict([token, True] for token in custom_tokens))
print(result)

Negative
